   1  : 1.58,
   0.1: 2.71,
  0.05: 3.84, 
  0.01: 6.63,
 0.005: 7.88, 
 0.001: 10.83

In [18]:
# 2.71 - 1.13

1.58

In [5]:
import pandas as pd
import numpy as np
from joblib import dump

from chi import square as cs

In [30]:
data = pd.read_excel("komentar/data_clean.xlsx")
komentar = data.komentar.tolist()
label = np.array(data.label.tolist())
y = label
data.head()

,Unnamed: 0,nomer,label,komentar
0,1668.0,0,non spam,waduh mara ken ae mbak via ni kirim balikpapan...
1,0.0,1,spam,awass akun adu domba min tu org suka nyinyirin...
2,1669.0,2,non spam,begini kalo ga biasa lipsing sekali lipsing ku...
3,1670.0,3,non spam,nitijen mha bebar bkin mlu 2 ajh kliaan apa sd...
4,1.0,4,spam,maantaap banggeet kaak eteaah akuu pakee gemuk...


# Ekstraksi dan Seleksi Fitur

In [31]:
sf = cs.ChiSquare(alpha = 0.1)
sf.find_best_features(komentar, label, fitur = None)

In [32]:
print("jumlah fitur terbaik:",len(sf.best_features),"dari",len(sf.fitur), "fitur")
best_features = sf.best_features

jumlah fitur terbaik: 2442 dari 14058 fitur


# Pemodelan

## Pemodelan TF-IDF

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(vocabulary = best_features)#
X = vectorizer.fit_transform(komentar)
save = vectorizer.fit(komentar)
dump(save, "model/tfidf.w")
# print(vectorizer.get_feature_names())
fitur = vectorizer.get_feature_names()
print(X.shape)

(4757, 2442)


## Pemodelan SVM

In [34]:
from sklearn.svm import SVC
clf = SVC()#gamma='auto'

# Pemodelan dan Validasi Model

In [35]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

start = 0.1
stop = 1
step = start
alpha_list = np.arange(start,stop+step,step)
print(alpha_list)
akurasi = list()

ix = 0 #just counter
for alpha in alpha_list:
    clf.set_params(gamma=alpha)
    temp_akurasi = list()
    for train_index, test_index in skf.split(X, y):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        ##Trainin Model menggunakan sebagian data latih (K-fold)
        clf.fit(X_train,y_train)
        
        #prediksi menggunakan data uji dari k-fold
        y_pred = clf.predict(X_test)
        temp_akurasi.append(accuracy_score(y_test, y_pred))
    rerata_akurasi = sum(temp_akurasi)/len(temp_akurasi)
    akurasi.append(rerata_akurasi)
    ix +=1
    if ix%100 !=0:
        print(".", end="")
    else:
#         print(alpha, ' | ', rerata_akurasi)
        print("")

        pass

ditt = {
    "gamma":alpha_list,
    "akurasi":akurasi
}
index = akurasi.index(max(akurasi))
acc = pd.DataFrame.from_dict(ditt)
acc

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
..........

,gamma,akurasi
0,0.1,0.931914
1,0.2,0.937378
2,0.3,0.938428
3,0.4,0.938428
4,0.5,0.939058
5,0.6,0.940108
6,0.7,0.940739
7,0.8,0.940949
8,0.9,0.940738
9,1.0,0.940107


In [36]:
print(alpha_list[index])
print(akurasi[index])

0.8
0.9409486266535744


In [27]:
print(alpha_list[index])
print(akurasi[index])

0.7000000000000001
0.9357948280003399


# Menyimpan Model

In [37]:
clf.set_params(gamma=alpha_list[index])

#Trainin Model menggunakan seluruh data latih
clf.fit(X, y)
dump(clf, "model/svm.m")

['model/svm.m']